In [5]:
import time
import json
import pandas as pd
import numpy as np
def get_last_data():
    import requests
    url = r"""https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&_={0}""".format(int(time.time()*1000))
    result = requests.get(url)
    data = json.loads(result.json()['data'])

    return data

def get_hist_data():
    import requests
    url = r"""https://view.inews.qq.com/g2/getOnsInfo?name=disease_other&_={0}""".format(int(time.time()*1000))
    result = requests.get(url)
    data = json.loads(result.json()['data'])

    return data

def get_world_data():
    import requests
    url = r"""https://view.inews.qq.com/g2/getOnsInfo?name=disease_foreign&_={0}""".format(int(time.time()*1000))
    result = requests.get(url)
    data = json.loads(result.json()['data'])

    return data

last_data = get_last_data()
world_data = get_world_data()

In [6]:
def get_city_last_info(data, province):
    for i, p in enumerate(data['areaTree'][0]['children']):
        if p['name'] == province:
            break
    today = list()
    total = list()

    for city in p['children']:
        city_today = city['today']
        city_today['city'] = city['name']
        city_today['province'] = province
        today.append(city_today)

    for city in p['children']:
        city_total = city['total']
        city_total['city'] = city['name']
        city_total['province'] = province
        total.append(city_total)

    return pd.DataFrame(today), pd.DataFrame(total)

zhejiang_today, zhejiang_total = get_city_last_info(last_data, '浙江')
anhui_today, anhui_total = get_city_last_info(last_data, '安徽')
hubei_today, hubei_total = get_city_last_info(last_data, '黑龙江')
shanghai_today, shanghai_total = get_city_last_info(last_data, '上海')
today = pd.concat((hubei_today, anhui_today, zhejiang_today, shanghai_today), axis=0)
today[today['province'] == '黑龙江']

,confirm,confirmCuts,isUpdated,city,province
0,0,0,False,地区待确认,黑龙江
1,0,0,False,哈尔滨,黑龙江
2,0,0,False,境外输入,黑龙江
3,0,0,False,牡丹江,黑龙江
4,0,0,False,大庆,黑龙江
5,0,0,False,齐齐哈尔,黑龙江
6,0,0,False,大兴安岭,黑龙江
7,0,0,False,黑河,黑龙江
8,0,0,False,双鸭山,黑龙江
9,0,0,False,绥化,黑龙江


In [7]:
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from pyecharts.charts import Line
hist_data = get_hist_data()
china_day_list = pd.DataFrame(hist_data['chinaDayList'])
china_day_list.head()

,confirm,suspect,dead,heal,nowConfirm,nowSevere,importedCase,deadRate,healRate,date,noInfect
0,41,0,1,0,0,0,0,2.4,0.0,01.13,0
1,41,0,1,0,0,0,0,2.4,0.0,01.14,0
2,41,0,2,5,0,0,0,4.9,12.2,01.15,0
3,45,0,2,8,0,0,0,4.4,17.8,01.16,0
4,62,0,2,12,0,0,0,3.2,19.4,01.17,0


In [8]:
# 1. 创建折线图对象，并使用WALDENz主题
line = Line(init_opts=opts.InitOpts(theme=ThemeType.WALDEN))

# 2. 添加x轴数据
line.add_xaxis(china_day_list['date'])

# 3. 添加y数据
line.add_yaxis('现有确诊', china_day_list['nowConfirm'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False,
              linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('累计确诊', china_day_list['confirm'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('疑似', china_day_list['suspect'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))

# 4. 全局配置
line.set_global_opts(
            title_opts=opts.TitleOpts(title="全国 现有确诊/疑似/累计确诊 趋势", subtitle="单位: 例"),
            yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(width=2, color="white")),
                                     splitline_opts=opts.SplitLineOpts(is_show=True)
                                    ),
            xaxis_opts=opts.AxisOpts(type_="category", axislabel_opts=opts.LabelOpts(rotate=45, font_family='Microsoft YaHei')),
            legend_opts=opts.LegendOpts(pos_right='right', pos_top='20%', legend_icon='circle', item_width=8),
            tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', background_color='white', border_color='black', axis_pointer_type='line', 
                                          border_width=1,textstyle_opts=opts.TextStyleOpts(font_size=14, color='black')))
line.render('diagnose.html')

'd:\\Program Op\\Jupyter\\diagnose.html'

In [9]:
line = Line(init_opts=opts.InitOpts(theme=ThemeType.WALDEN))

line.add_xaxis(china_day_list['date'])
line.add_yaxis('治愈', china_day_list['heal'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False,
              linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('死亡', china_day_list['dead'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))
line.set_global_opts(
            title_opts=opts.TitleOpts(title="全国 累计治愈/死亡 趋势", subtitle="单位: 例"),
            yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(width=2, color="white")),
                                     splitline_opts=opts.SplitLineOpts(is_show=True)
                                    ),
            xaxis_opts=opts.AxisOpts(type_="category", axislabel_opts=opts.LabelOpts(rotate=45, font_family='Microsoft YaHei')),
            legend_opts=opts.LegendOpts(pos_right='right', pos_top='20%', legend_icon='circle', item_width=8),
            tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', background_color='white', border_color='black', axis_pointer_type='line', 
                                          border_width=1,textstyle_opts=opts.TextStyleOpts(font_size=14, color='black')))
line.render('healing.html')

'd:\\Program Op\\Jupyter\\healing.html'

In [10]:
china_day_add_list = pd.DataFrame(hist_data['chinaDayAddList'])

line = Line(init_opts=opts.InitOpts(theme=ThemeType.WALDEN))

line.add_xaxis(china_day_add_list['date'])
line.add_yaxis('新增治愈', china_day_add_list['heal'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False,
              linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('新增确诊', china_day_add_list['confirm'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('新增死亡', china_day_add_list['dead'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))
line.add_yaxis('新增疑似', china_day_add_list['suspect'], label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False, 
               linestyle_opts=opts.LineStyleOpts(width=2))
line.set_global_opts(
            title_opts=opts.TitleOpts(title="全国 新增确诊/疑似/治愈/死亡 趋势", subtitle="单位: 例"),
            yaxis_opts=opts.AxisOpts(axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(width=2, color="white")),
                                     splitline_opts=opts.SplitLineOpts(is_show=True)
                                    ),
            xaxis_opts=opts.AxisOpts(type_="category", axislabel_opts=opts.LabelOpts(rotate=45, font_family='Microsoft YaHei')),
            legend_opts=opts.LegendOpts(pos_right='right', pos_top='20%', legend_icon='circle', item_width=8),
            tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', background_color='white', border_color='black', axis_pointer_type='line', 
                                          border_width=1,textstyle_opts=opts.TextStyleOpts(font_size=14, color='black')))
line.render('death.html')

'd:\\Program Op\\Jupyter\\death.html'

In [11]:
from pyecharts import options as opts
from pyecharts.charts import Map
confirms = list()
for province in last_data['areaTree'][0]['children']:
    province_name = province['name']
    province_data = province['total']['confirm'] - province['total']['heal'] - province['total']['dead']
    confirms.append([province_name, province_data])
map_ = Map()
map_.add("地区", confirms, "china", is_map_symbol_show=False)
map_.set_global_opts(title_opts=opts.TitleOpts(title="当前现有确诊病例数，排除治愈、死亡"), 
                     visualmap_opts=opts.VisualMapOpts(max_=40000, is_piecewise=True,
                                                       pieces=[
                                                           {"min": 10000},
                                                           {"min": 3000, "max": 10000}, 
                                                           {"min": 1500, "max": 3000},
                                                           {"min": 800, "max": 1500},
                                                           {"min": 400, "max": 800},
                                                           {"min": 200, "max": 400},
                                                           {"min": 100, "max": 200},
                                                           {"min": 60, "max": 100},
                                                           {"min": 30, "max": 60},
                                                           {"min": 5, "max": 30},
                                                           {"max": 5}
                                                          ],
#                                                        range_color
                                                      ))
map_.render('china.html')

'd:\\Program Op\\Jupyter\\china.html'

In [13]:
# 国家中英文对照，完整从：
# https://blog.csdn.net/u012557538/article/details/78490267?utm_source=blogxgwz1
nameMap = {
    'Afghanistan':'阿富汗',
    'Singapore':'新加坡',
    'Angola':'安哥拉',
    'Albania':'阿尔巴尼亚',
    'United Arab Emirates':'阿联酋',
    'Argentina':'阿根廷',
    'Armenia':'亚美尼亚',
    'French Southern and Antarctic Lands':'法属南半球和南极领地',
    'Australia':'澳大利亚',
    'Austria':'奥地利',
    'Azerbaijan':'阿塞拜疆',
    'Burundi':'布隆迪',
    'Belgium':'比利时',
    'Benin':'贝宁',
    'Burkina Faso':'布基纳法索',
    'Bangladesh':'孟加拉国',
    'Bulgaria':'保加利亚',
    'The Bahamas':'巴哈马',
    'Bosnia and Herzegovina':'波斯尼亚和黑塞哥维那',
    'Belarus':'白俄罗斯',
    'Belize':'伯利兹',
    'Bermuda':'百慕大',
    'Bolivia':'玻利维亚',
    'Brazil':'巴西',
    'Brunei':'文莱',
    'Bhutan':'不丹',
    'Botswana':'博茨瓦纳',
    'Central African Republic':'中非共和国',
    'Canada':'加拿大',
    'Switzerland':'瑞士',
    'Chile':'智利',
    'China':'中国',
    'Ivory Coast':'象牙海岸',
    'Cameroon':'喀麦隆',
    'Democratic Republic of the Congo':'刚果民主共和国',
    'Republic of the Congo':'刚果共和国',
    'Colombia':'哥伦比亚',
    'Costa Rica':'哥斯达黎加',
    'Cuba':'古巴',
    'Northern Cyprus':'北塞浦路斯',
    'Cyprus':'塞浦路斯',
    'Czech Republic':'捷克共和国',
    'Germany':'德国',
    'Djibouti':'吉布提',
    'Denmark':'丹麦',
    'Dominican Republic':'多明尼加共和国',
    'Algeria':'阿尔及利亚',
    'Ecuador':'厄瓜多尔',
    'Egypt':'埃及',
    'Eritrea':'厄立特里亚',
    'Spain':'西班牙',
    'Estonia':'爱沙尼亚',
    'Ethiopia':'埃塞俄比亚',
    'Finland':'芬兰',
    'Fiji':'斐',
    'Falkland Islands':'福克兰群岛',
    'France':'法国',
    'Gabon':'加蓬',
    'United Kingdom':'英国',
    'Georgia':'格鲁吉亚',
    'Ghana':'加纳',
    'Guinea':'几内亚',
    'Gambia':'冈比亚',
    'Guinea Bissau':'几内亚比绍',
    'Equatorial Guinea':'赤道几内亚',
    'Greece':'希腊',
    'Greenland':'格陵兰',
    'Guatemala':'危地马拉',
    'French Guiana':'法属圭亚那',
    'Guyana':'圭亚那',
    'Honduras':'洪都拉斯',
    'Croatia':'克罗地亚',
    'Haiti':'海地',
    'Hungary':'匈牙利',
    'Indonesia':'印尼',
    'India':'印度',
    'Ireland':'爱尔兰',
    'Iran':'伊朗',
    'Iraq':'伊拉克',
    'Iceland':'冰岛',
    'Israel':'以色列',
    'Italy':'意大利',
    'Jamaica':'牙买加',
    'Jordan':'约旦',
    'Japan':'日本',
    'Kazakhstan':'哈萨克斯坦',
    'Kenya':'肯尼亚',
    'Kyrgyzstan':'吉尔吉斯斯坦',
    'Cambodia':'柬埔寨',
    'South Korea':'韩国',
    'Kosovo':'科索沃',
    'Kuwait':'科威特',
    'Laos':'老挝',
    'Lebanon':'黎巴嫩',
    'Liberia':'利比里亚',
    'Libya':'利比亚',
    'Sri Lanka':'斯里兰卡',
    'Lesotho':'莱索托',
    'Lithuania':'立陶宛',
    'Luxembourg':'卢森堡',
    'Latvia':'拉脱维亚',
    'Morocco':'摩洛哥',
    'Moldova':'摩尔多瓦',
    'Madagascar':'马达加斯加',
    'Mexico':'墨西哥',
    'Macedonia':'马其顿',
    'Mali':'马里',
    'Myanmar':'缅甸',
    'Montenegro':'黑山',
    'Mongolia':'蒙古',
    'Mozambique':'莫桑比克',
    'Mauritania':'毛里塔尼亚',
    'Malawi':'马拉维',
    'Malaysia':'马来西亚',
    'Namibia':'纳米比亚',
    'New Caledonia':'新喀里多尼亚',
    'Niger':'尼日尔',
    'Nigeria':'尼日利亚',
    'Nicaragua':'尼加拉瓜',
    'Netherlands':'荷兰',
    'Norway':'挪威',
    'Nepal':'尼泊尔',
    'New Zealand':'新西兰',
    'Oman':'阿曼',
    'Pakistan':'巴基斯坦',
    'Panama':'巴拿马',
    'Peru':'秘鲁',
    'Philippines':'菲律宾',
    'Papua New Guinea':'巴布亚新几内亚',
    'Poland':'波兰',
    'Puerto Rico':'波多黎各',
    'North Korea':'北朝鲜',
    'Portugal':'葡萄牙',
    'Paraguay':'巴拉圭',
    'Qatar':'卡塔尔',
    'Romania':'罗马尼亚',
    'Russia':'俄罗斯',
    'Rwanda':'卢旺达',
    'Western Sahara':'西撒哈拉',
    'Saudi Arabia':'沙特阿拉伯',
    'Sudan':'苏丹',
    'South Sudan':'南苏丹',
    'Senegal':'塞内加尔',
    'Solomon Islands':'所罗门群岛',
    'Sierra Leone':'塞拉利昂',
    'El Salvador':'萨尔瓦多',
    'Somaliland':'索马里兰',
    'Somalia':'索马里',
    'Republic of Serbia':'塞尔维亚',
    'Suriname':'苏里南',
    'Slovakia':'斯洛伐克',
    'Slovenia':'斯洛文尼亚',
    'Sweden':'瑞典',
    'Swaziland':'斯威士兰',
    'Syria':'叙利亚',
    'Chad':'乍得',
    'Togo':'多哥',
    'Thailand':'泰国',
    'Tajikistan':'塔吉克斯坦',
    'Turkmenistan':'土库曼斯坦',
    'East Timor':'东帝汶',
    'Trinidad and Tobago':'特里尼达和多巴哥',
    'Tunisia':'突尼斯',
    'Turkey':'土耳其',
    'United Republic of Tanzania':'坦桑尼亚',
    'Uganda':'乌干达',
    'Ukraine':'乌克兰',
    'Uruguay':'乌拉圭',
    'United States':'美国',
    'Uzbekistan':'乌兹别克斯坦',
    'Venezuela':'委内瑞拉',
    'Vietnam':'越南',
    'Vanuatu':'瓦努阿图',
    'West Bank':'西岸',
    'Yemen':'也门',
    'South Africa':'南非',
    'Zambia':'赞比亚',
    'Zimbabwe':'津巴布韦'
}


name_map_ = {value: key for key, value in nameMap.items()}
name_map_['日本本土'] = 'Japan'
name_map_['印度尼西亚'] = 'Indonesia'
name_map_['捷克'] = 'Czech Republic'

world_confirms = list()
for conuntry in world_data['foreignList']:
    try:
        conuntry_name = name_map_[conuntry['name']]
        conuntry_data = conuntry['confirm'] - conuntry['heal'] - conuntry['dead']
        world_confirms.append([conuntry_name, conuntry_data])
    except Exception as e:
        print(conuntry['name'])
        
world_confirms.append(['China', 129])

map_ = Map()
map_.add("国家", world_confirms, "world", is_map_symbol_show=False)
map_.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
map_.set_global_opts(title_opts=opts.TitleOpts(title="世界各国当前现有确诊病例数，排除治愈、死亡"), 
                     visualmap_opts=opts.VisualMapOpts(max_=40000, is_piecewise=True,
                                                       pieces=[
                                                           {"min": 10000},
                                                           {"min": 3000, "max": 10000}, 
                                                           {"min": 1500, "max": 3000},
                                                           {"min": 800, "max": 1500},
                                                           {"min": 400, "max": 800},
                                                           {"min": 200, "max": 400},
                                                           {"min": 100, "max": 200},
                                                           {"min": 60, "max": 100},
                                                           {"min": 30, "max": 60},
                                                           {"min": 5, "max": 30},
                                                           {"max": 5}
                                                          ],
#                                                        range_color
                                                      ))
map_.render('woldwile.html')

多米尼加
钻石号邮轮
巴林
安道尔
波黑
北马其顿
圣马力诺
科特迪瓦
马耳他
毛里求斯
巴勒斯坦
刚果（金）
列支敦士登公国
孟加拉
摩纳哥
刚果（布）
马尔代夫
塞舌尔
安提瓜和巴布达
佛得角
梵蒂冈


'd:\\Program Op\\Jupyter\\woldwile.html'